In [ ]:
# default_exp preprocessing.vocab

In [ ]:
#all_slow

# Vocab
> Classes and functions to create vocabs from cleaned EHR data.

In [ ]:
#hide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#export
from lemonpie.basics import *
from lemonpie.preprocessing.clean import *
from fastai.imports import *
from datetime import date

In [ ]:
#hide
from nbdev.showdoc import *

## `nn.Embedding` and `nn.EmbeddingBag`

**nn.Embedding**

In [ ]:
emb1 = nn.Embedding(5,3)

In [ ]:
emb1(torch.LongTensor([[0,1,2,3,4]]))

tensor([[[ 0.1523, -0.3658,  1.1193],
         [ 0.0235, -0.1970,  0.2325],
         [ 0.7203,  2.3237, -0.1296],
         [ 0.5819, -0.2482, -0.3753],
         [ 0.1162,  0.5788, -0.3371]]], grad_fn=<EmbeddingBackward>)

An embedding matrix is a lookup table
1. `emb1` above has 5 rows, that is 5 elements
2. but looking up an element, returns a vector for that element.

Given this embedding matrix, looking up elements 1, 2, 4 will look like this .. 

In [ ]:
input = torch.LongTensor([[1,2,4]])

In [ ]:
emb1(input)

tensor([[[ 0.0235, -0.1970,  0.2325],
         [ 0.7203,  2.3237, -0.1296],
         [ 0.1162,  0.5788, -0.3371]]], grad_fn=<EmbeddingBackward>)

Batch of inputs is also possible (in this case a batch of 2, each with 3 elements being looked up) 
- Note that inputs (# of elements being looked up) in a batch have to be of the same size

In [ ]:
input = torch.LongTensor([[1,2,4],[0,3,2]])
# input = torch.LongTensor([[1,2,4],[0,3,2,1]]) # this will fail

In [ ]:
emb1(input)

tensor([[[ 0.0235, -0.1970,  0.2325],
         [ 0.7203,  2.3237, -0.1296],
         [ 0.1162,  0.5788, -0.3371]],

        [[ 0.1523, -0.3658,  1.1193],
         [ 0.5819, -0.2482, -0.3753],
         [ 0.7203,  2.3237, -0.1296]]], grad_fn=<EmbeddingBackward>)

**`nn.EmbeddingBag`**

In [ ]:
embg1 = nn.EmbeddingBag(5,3)

Exactly the same input as in case of `nn.Embedding` above (batch of 2)
- but the result will be averaged across the 3 elements in a batch
- resulting in an output of 2 vectors not 6 like above

In [ ]:
input = torch.LongTensor([[1,2,4],[0,3,2]]) # exactly same as above, but o/p is avg'd now

In [ ]:
embg1(input)

tensor([[ 0.3981, -0.1970, -0.5963],
        [ 0.5032,  0.6250,  0.0026]], grad_fn=<EmbeddingBagBackward>)

Another way to do this is to send in `offsets` rather than separating the inputs into 2 (or x number of) lists

In [ ]:
input = torch.LongTensor([1,2,4,0,3,2]) #same as above - 2 of same length 3
offsets = torch.LongTensor([0,3]) # output will be avg'd by default

In [ ]:
embg1(input, offsets)

tensor([[ 0.3981, -0.1970, -0.5963],
        [ 0.5032,  0.6250,  0.0026]], grad_fn=<EmbeddingBagBackward>)

In [ ]:
input = torch.LongTensor([1,2,4,2,0,3,3,2]) #same as above - batch of 2 inputs, but of length 4 each
offsets = torch.LongTensor([0,4])

In [ ]:
embg1(input, offsets) #avg'd 2 outputs one for each input batch i.e. avg'd across 4 in each batch

tensor([[ 0.4415, -0.0460, -0.8777],
        [ 0.3911,  0.8386,  0.4091]], grad_fn=<EmbeddingBagBackward>)

**Different Sizes**

Offsets allow us to have input batches of different lengths

In [ ]:
input = torch.LongTensor([1,2,4,2,0,3,3,2]) #same input as above but .. 
offsets = torch.LongTensor([0,3,5]) #this indicates - 3 batches of different lengths (0,1,2)(3,4)(5,6,7)

In [ ]:
embg1(input, offsets)

tensor([[ 0.3981, -0.1970, -0.5963],
        [ 0.7276,  0.1977, -0.8106],
        [ 0.2269,  1.1220,  0.5119]], grad_fn=<EmbeddingBagBackward>)

**Application to EHR Data**

TODO - Details

## `itoc`, `ctoi`, `ctod`, `numericalize`, `textify`

These have the same meaning as in fastai v1
- `itoc` = index to code
- `ctoi` = the reverse of `itoc`
- `ctod` = is a new addition - to show descriptions in case descriptions exist in some type of EHR codes


- `numericalize()` = returns numericalized ids (`ctoi`) for a set of codes
- `textify()` = reverse of `numericalize()` - returns the codes (`itoc`) for a set of numericalized ids and if descriptions exist return them too (`ctod`)

I tried to extend fastai vocabs, but found it easier to write from scratch, given EHR data is quite unique.

## Vocabs

In [ ]:
#clean_raw_ehrdata(PATH_1K, today=SYNTHEA_DATAGEN_DATES['1K'])

In [ ]:
code_dfs = load_ehr_vocabcodes(PATH_1K)

In [ ]:
pt_codes, obs_codes, alg_codes, crpl_codes, med_codes, img_codes, proc_codes, cnd_codes, immn_codes = code_dfs

The following convenience function returns 2 results in order to make the given input a multiple of 8, to make it Tensor Core friendly for mixed precision training.
1. `pad` - the extra padding needed to round up to the next multiple of 8.
    - will be used during vocab creation to increase the vocab size **(round up)** with dummy padding elements.
2. `opt` - for optimal - the number needed to round up or round down to the nearest multiple of 8 with a preference for rounding down.
    - will be used when calculating the width for an embedding matrix based on a vocab.
    - rounding up or down can be used to figure out **nearest** multiple of 8.

In [ ]:
#export
def multiple_of_8(orig):
    '''Return extra padding or optimization needed to make multiple of 8 for AMP.'''
    mod = orig % 8
    if mod == 0:
        pad = opt = 0
    else:
        pad = 8 - mod
        opt = -mod if mod <= 4 else (8-mod)

    return pad, opt

In [ ]:
print(f'number -- (pad, optimum)')
for i in range(16, 25):
    print(f'{i} -- {multiple_of_8(i)}')

number -- (pad, optimum)
16 -- (0, 0)
17 -- (7, -1)
18 -- (6, -2)
19 -- (5, -3)
20 -- (4, -4)
21 -- (3, 3)
22 -- (2, 2)
23 -- (1, 1)
24 -- (0, 0)


In [ ]:
#export
class EhrVocab():
    '''Vocab class for most EHR datatypes'''
    def __init__(self, itoc, ctoi, ctod=None):
        self.itoc = itoc
        self.ctoi = ctoi
        if ctod is not None: self.ctod = ctod 
        self.vocab_size = len(self.itoc)
        
    @classmethod
    def create(cls, codes_df):
        '''Create vocab object (itoc, ctoi and maybe ctod) from the codes df'''
        desc_exists = 'desc' in codes_df.columns
        codes_df = codes_df.astype({'code':'str'})
        itoc = list(codes_df.code.unique())  #old --> list(set(codes_df.code))
        itoc.insert(0,'xxnone')
        itoc.insert(1,'xxunk')
        
        orig_len = len(itoc)
        amp_pad_sz, _ = multiple_of_8(orig_len)
        itoc.extend(['xxamp' for _ in range(amp_pad_sz)])
        
        ctoi = {code: i for i, code in enumerate(itoc)}
        
        if desc_exists:
            codes_df.set_index('code', inplace=True)
            ctod = {}
            ctod[itoc[0]] = "Nothing recorded"
            ctod[itoc[1]] = "Unknown"
            for code in itoc[2:orig_len]: 
                ctod[code] = set(codes_df.loc[code].desc)
            for code in itoc[orig_len:]:
                ctod[code] = "Padding for AMP"
        
        return cls(itoc, ctoi, ctod) if desc_exists else cls(itoc, ctoi)
    
    def get_emb_dims(self, αd=0.5736):
        '''Get embedding dimensions'''
        width = round(6 * αd * (self.vocab_size**0.25))
        _, amp_optimum = multiple_of_8(width)
        width += amp_optimum
        return self.vocab_size, width
    
    def numericalize(self, codes, log_excep=LOG_NUMERICALIZE_EXCEP, log_dir='default_log_store'):
        '''Lookup and return indices for codes'''
        
        if log_excep:
            today = date.today().strftime("%Y-%m-%d")
            log_dir = LOG_STORE if log_dir=='default_log_store' else log_dir
            if not os.path.isdir(log_dir): os.mkdir(log_dir)
            logfile = f'{log_dir}/{today}_numericalize_exceptions.log'
                    
        res = []
        try:
            res = [self.ctoi[str(code)] for code in codes] #no big performance benefit
        except KeyError:
            for code in codes:
                try:
                    res.append(self.ctoi[str(code)])
                except KeyError:
                    res.append(self.ctoi['xxunk'])
                    if log_excep:
                        with open(logfile, 'a') as log:
                            log.write(f'\ncode: {code}')                      
                    
        return res
    
    def textify(self, indxs):
        '''Lookup and return descriptions for codes'''
        if hasattr(self, 'ctod'):
            res = [ (self.itoc[i], self.ctod[self.itoc[i]]) for i in indxs ]
        else:
            res = [ (self.itoc[i]) for i in indxs ]
        return res

In [ ]:
show_doc(EhrVocab, title_level=3)

<h3 id="EhrVocab" class="doc_header"><code>class</code> <code>EhrVocab</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>EhrVocab</code>(**`itoc`**, **`ctoi`**, **`ctod`**=*`None`*)

Vocab class for most EHR datatypes

In [ ]:
show_doc(EhrVocab.create)

<h4 id="EhrVocab.create" class="doc_header"><code>EhrVocab.create</code><a href="__main__.py#L10" class="source_link" style="float:right">[source]</a></h4>

> <code>EhrVocab.create</code>(**`codes_df`**)

Create vocab object (itoc, ctoi and maybe ctod) from the codes df

In [ ]:
show_doc(EhrVocab.numericalize)

<h4 id="EhrVocab.numericalize" class="doc_header"><code>EhrVocab.numericalize</code><a href="__main__.py#L44" class="source_link" style="float:right">[source]</a></h4>

> <code>EhrVocab.numericalize</code>(**`codes`**, **`log_excep`**=*`True`*, **`log_dir`**=*`'default_log_store'`*)

Lookup and return indices for codes

In [ ]:
show_doc(EhrVocab.textify)

<h4 id="EhrVocab.textify" class="doc_header"><code>EhrVocab.textify</code><a href="__main__.py#L68" class="source_link" style="float:right">[source]</a></h4>

> <code>EhrVocab.textify</code>(**`indxs`**)

Lookup and return descriptions for codes

In [ ]:
show_doc(EhrVocab.get_emb_dims)

<h4 id="EhrVocab.get_emb_dims" class="doc_header"><code>EhrVocab.get_emb_dims</code><a href="__main__.py#L37" class="source_link" style="float:right">[source]</a></h4>

> <code>EhrVocab.get_emb_dims</code>(**`αd`**=*`0.5736`*)

Get embedding dimensions

In [ ]:
#export
class ObsVocab (EhrVocab):
    '''Special Vocab class for Observation codes'''
    def __init__(self, vocab_df):
        self.vocab_df = vocab_df
        self.vocab_size = len(vocab_df)
    
    def numericalize(self, codes, log_excep=LOG_NUMERICALIZE_EXCEP, log_dir='default_log_store'):
        '''Numericalize observation codes (return indices for codes)'''
        
        if log_excep:
            today = date.today().strftime("%Y-%m-%d")
            log_dir = LOG_STORE if log_dir=='default_log_store' else log_dir
            if not os.path.isdir(log_dir): os.mkdir(log_dir)
            logfile = f'{log_dir}/{today}_numericalize_exceptions.log'
        
        indxs = []
        for code in codes:
            if code in ['xxnone','xxunk']: indxs.extend(self.vocab_df[(self.vocab_df['code'] == code)].index.tolist())
            else: 
                c,v,u,t = code.split('||')
                if t == 'numeric':
                    filt_df = self.vocab_df[(self.vocab_df['code'] == c) & (self.vocab_df['units'] == u) & (self.vocab_df['type'] == t)]
                    res = filt_df.iloc[(filt_df.value - float(v)).abs().argsort()[:1]].index.tolist()
                else:
                    res = self.vocab_df[(self.vocab_df['code'] == c) & (self.vocab_df['value'] == v) & \
                                               (self.vocab_df['units'] == u) & (self.vocab_df['type'] == t)].index.tolist()
                if len(res) == 0: 
                    indxs.extend(self.vocab_df[(self.vocab_df['code'] == 'xxunk')].index.tolist())
                    if log_excep:
                        with open(logfile, 'a') as log:
                            log.write(f'\ncode in ObsVocab: {code}')                    
                else            : indxs.extend(res)
                    
        assert len(codes) == len(indxs), "Possible bug, not all codes being numericalized"
        return indxs
    
    def textify(self, indxs):
        '''Textify observation codes (returns codes and descriptions)'''
        txts = []
        for i in indxs:
            c,d,v,u,t = self.vocab_df.iloc[i]
            if i == 0: txts.append((c, d))
            else:      txts.append((f'{c}||{v}||{u}||{t}', d))
        assert len(indxs) == len(txts), "Possible bug, not all indxs being textified"
        return txts

    @classmethod
    def create(cls, obs_codes, num_buckets=5):
        '''Create vocab object from observation codes'''
        numerics = pd.DataFrame(obs_codes.loc[obs_codes['type'] == 'numeric',:])
        texts = pd.DataFrame(obs_codes.loc[obs_codes['type'] == 'text',:])
        numerics = numerics.astype({'value':'float'}, copy=False)
        vocab_rows = []

        for code in numerics.orig_code.unique():
            this_code = numerics.loc[numerics['orig_code'] == code]
            for unit in this_code.units.unique():
                this_unit = this_code.loc[this_code['units'] == unit]
                for val in np.linspace(this_unit.value.min(), this_unit.value.max(), num=num_buckets):
                    vocab_rows.append([code,this_unit.desc.iloc[0],val,unit,'numeric'])

        for code in texts.orig_code.unique():
            this_code = texts.loc[texts['orig_code'] == code]
            for unit in this_code.units.unique():
                this_unit = this_code.loc[this_code['units'] == unit]
                for val in this_unit.value.unique():
                    vocab_rows.append([code,this_unit.desc.iloc[0],val,unit,'text'])
                    
        vocab_rows.insert(0, ['xxnone','Nothing recorded','xxnone','xxnone','xxnone'])
        vocab_rows.insert(1, ['xxunk','Unknown','xxunk','xxunk','xxunk'])
        amp_pad_sz, _ = multiple_of_8(len(vocab_rows)) 
        for _ in range(amp_pad_sz):
            vocab_rows.append(['xxamp','Padding for AMP','xxamp','xxamp','xxamp'])
        
        obs_vocab = pd.DataFrame(data=vocab_rows, columns=['code','desc','value','units','type'])
        assert obs_codes.orig_code.nunique() == obs_vocab.code.nunique() - 3, "Possible bug, obs_code nuniques don't match"
        
        return cls(obs_vocab)

In [ ]:
show_doc(ObsVocab, title_level=3)

<h3 id="ObsVocab" class="doc_header"><code>class</code> <code>ObsVocab</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>ObsVocab</code>(**`vocab_df`**) :: [`EhrVocab`](/lemonpie/preprocessing_vocab.html#EhrVocab)

Special Vocab class for Observation codes

In [ ]:
show_doc(ObsVocab.create)

<h4 id="ObsVocab.create" class="doc_header"><code>ObsVocab.create</code><a href="__main__.py#L48" class="source_link" style="float:right">[source]</a></h4>

> <code>ObsVocab.create</code>(**`obs_codes`**, **`num_buckets`**=*`5`*)

Create vocab object from observation codes

In [ ]:
show_doc(ObsVocab.numericalize)

<h4 id="ObsVocab.numericalize" class="doc_header"><code>ObsVocab.numericalize</code><a href="__main__.py#L8" class="source_link" style="float:right">[source]</a></h4>

> <code>ObsVocab.numericalize</code>(**`codes`**, **`log_excep`**=*`True`*, **`log_dir`**=*`'default_log_store'`*)

Numericalize observation codes (return indices for codes)

- split incoming concated `code||value||units||type` string
- get a result_df based on everything except value
- then do an `argsort()` on the value column to determine closest value
 - based on example given in [pandas docs - cookbook](https://pandas.pydata.org/docs/user_guide/cookbook.html#building-criteria)
   - **cookbook example that uses `loc` doesnt work, instead `iloc` [works](https://stackoverflow.com/questions/30112202/how-do-i-find-the-closest-values-in-a-pandas-series-to-an-input-number/53553226)**
 - `argsort()` - [Returns the indices that would sort this array](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html#numpy.argsort)
 - `[:1]` on that returns the one row with the closest match, index of that is what we want

In [ ]:
show_doc(ObsVocab.textify)

<h4 id="ObsVocab.textify" class="doc_header"><code>ObsVocab.textify</code><a href="__main__.py#L38" class="source_link" style="float:right">[source]</a></h4>

> <code>ObsVocab.textify</code>(**`indxs`**)

Textify observation codes (returns codes and descriptions)

> Note: During a pre-processing run, the default behavior is to log any numericalize errors that are encountered. This is controlled by the global-level variable `LOG_NUMERICALIZE_EXCEP` which is set to true by default.

> Important: `create()` and `get_emb_dims()` methods have logic in them to create vocab lengths and widths respectively that are Tensor Core-friendly, (i.e. multiples of 8) to help with mixed precision training.

In [ ]:
obs_codes.head()

,orig_code,desc,value,units,type
indx,,,,,
0,8302-2,Body Height,169.6,cm,numeric
1,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,4.0,{score},numeric
2,29463-7,Body Weight,63.8,kg,numeric
3,39156-5,Body Mass Index,22.2,kg/m2,numeric
4,59576-9,Body mass index (BMI) [Percentile] Per age and...,81.9,%,numeric


In [ ]:
obs_vocab_obj = ObsVocab.create(obs_codes)

In [ ]:
obs_vocab_obj.numericalize(['8302-2||200.3||cm||numeric', \
                            '72514-3||4||{score}||numeric', '33756-8||21.7||mm||numeric','29463-7||181.8||kg||numeric'])

[6, 9, 341, 16]

In [ ]:
#Testing unkown code
obs_vocab_obj.numericalize(['blah-2||200.3||cm||numeric', \
                            '72514-3||4||{score}||numeric', '33756-8||21.7||mm||numeric','29463-7||181.8||kg||numeric'])

[1, 9, 341, 16]

In [ ]:
obs_vocab_obj.textify([5, 8, 200, 15])

[('8302-2||160.29999999999998||cm||numeric', 'Body Height'),
 ('72514-3||2.5||{score}||numeric',
  'Pain severity - 0-10 verbal numeric rating [Score] - Reported'),
 ('20565-8||26.75||mmol/L||numeric', 'Carbon Dioxide'),
 ('29463-7||98.09999999999998||kg||numeric', 'Body Weight')]

In [ ]:
obs_vocab_obj.numericalize(['32465-7||Normal size prostate||{nominal}||text',"80271-0||Positive Murphy's Sign||xxxnan||text",\
                          'xxnone'])

[522, 1, 0]

In [ ]:
obs_vocab_obj.vocab_size

536

In [ ]:
obs_vocab_obj.textify([522, 523, 0])

[('32465-7||Normal size prostate||{nominal}||text',
  'Physical findings of Prostate'),
 ('32465-7||Prostate enlarged on PR||{nominal}||text',
  'Physical findings of Prostate'),
 ('xxnone', 'Nothing recorded')]

In [ ]:
obs_vocab_obj.numericalize(['xxnone','xxunk','72166-2||Never smoker||xxxnan||text'])

[0, 1, 467]

In [ ]:
obs_vocab_obj.textify([0, 1, 2, 3, 467, 497])

[('xxnone', 'Nothing recorded'),
 ('xxunk||xxunk||xxunk||xxunk', 'Unknown'),
 ('8302-2||45.1||cm||numeric', 'Body Height'),
 ('8302-2||83.5||cm||numeric', 'Body Height'),
 ('72166-2||Never smoker||xxxnan||text', 'Tobacco smoking status NHIS'),
 ('88040-1||Improving (qualifier value)||xxxnan||text',
  'Response to cancer treatment')]

## VocabList

In [ ]:
#export
class EhrVocabList:
    '''Class to create and hold all vocab objects for an entire dataset'''
    def __init__(self, demographics_vocabs, records_vocabs, age_mean, age_std, path):
        self.demographics_vocabs, self.records_vocabs, self.path = demographics_vocabs, records_vocabs, path
        self.age_mean, self.age_std = age_mean, age_std
    
    @classmethod
    def create(cls, path, num_buckets=5):
        '''Read all code dfs from the dataset path and create all vocab objects'''
        demographics_vocabs, records_vocabs = [], []
        code_dfs = load_ehr_vocabcodes(path)
        
        def _get_demographics_codes(pt_codes):
            code_dfs = []
            code_dfs.extend([pd.DataFrame(range(1, 32, 1), columns=['code'])]) #31 days  
            code_dfs.extend([pd.DataFrame(range(1, 13, 1), columns=['code'])]) #12 months 
            code_dfs.extend([pd.DataFrame(range(1900, pd.Timestamp.today().year + 1, 1), columns=['code'])]) #years 1900 to now
            code_dfs.extend([pd.DataFrame(pt_codes.marital.dropna().unique(), columns=['code'])])
            code_dfs.extend([pd.DataFrame(pt_codes.race.dropna().unique(), columns=['code'])])
            code_dfs.extend([pd.DataFrame(pt_codes.ethnicity.dropna().unique(), columns=['code'])])
            code_dfs.extend([pd.DataFrame(pt_codes.gender.dropna().unique(), columns=['code'])])
            code_dfs.extend([pd.DataFrame(pt_codes.birthplace.dropna().unique(), columns=['code'])])
            code_dfs.extend([pd.DataFrame(pt_codes.city.dropna().unique(), columns=['code'])])
            code_dfs.extend([pd.DataFrame(pt_codes.state.dropna().unique(), columns=['code'])])
            code_dfs.extend([pd.DataFrame(pt_codes.zip.dropna().unique(), columns=['code'])])
            age_mean, age_std = pt_codes.age_now_days.mean(), pt_codes.age_now_days.std()
            return code_dfs, age_mean, age_std
        
        demographics_codes, age_mean, age_std = _get_demographics_codes(code_dfs[0])
        demographics_vocabs.extend([EhrVocab.create(codes_df) for codes_df in demographics_codes])
        records_vocabs.extend([ObsVocab.create(code_dfs[1], num_buckets)])
        records_vocabs.extend([EhrVocab.create(codes_df) for codes_df in code_dfs[2:]])
        return cls(demographics_vocabs, records_vocabs, age_mean, age_std, path)    
    
    def save(self):
        '''Save vocablist (containing all vocab objects for the dataset)'''
        pckl_dir = Path(f'{self.path}/processed')
        pckl_dir.mkdir(parents=True, exist_ok=True)
        pckl_f = open(f'{pckl_dir}/vocabs.vocablist', 'wb')
        pickle.dump(self, pckl_f)
        pckl_f.close()
        print(f'Saved vocab lists to {pckl_dir}')
        
    @classmethod
    def load(cls, path):
        '''Load previously created vocablist object (containing all vocab objects for the dataset)'''
        infile = open(f'{path}/processed/vocabs.vocablist','rb')
        ehrVocabList = pickle.load(infile)
        infile.close()
        return ehrVocabList

In [ ]:
show_doc(EhrVocabList, title_level=3)

<h3 id="EhrVocabList" class="doc_header"><code>class</code> <code>EhrVocabList</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>EhrVocabList</code>(**`demographics_vocabs`**, **`records_vocabs`**, **`age_mean`**, **`age_std`**, **`path`**)

Class to create and hold all vocab objects for an entire dataset

In [ ]:
show_doc(EhrVocabList.create)

<h4 id="EhrVocabList.create" class="doc_header"><code>EhrVocabList.create</code><a href="__main__.py#L8" class="source_link" style="float:right">[source]</a></h4>

> <code>EhrVocabList.create</code>(**`path`**, **`num_buckets`**=*`5`*)

Read all code dfs from the dataset path and create all vocab objects

In [ ]:
show_doc(EhrVocabList.save)

<h4 id="EhrVocabList.save" class="doc_header"><code>EhrVocabList.save</code><a href="__main__.py#L36" class="source_link" style="float:right">[source]</a></h4>

> <code>EhrVocabList.save</code>()

Save vocablist (containing all vocab objects for the dataset)

In [ ]:
show_doc(EhrVocabList.load)

<h4 id="EhrVocabList.load" class="doc_header"><code>EhrVocabList.load</code><a href="__main__.py#L45" class="source_link" style="float:right">[source]</a></h4>

> <code>EhrVocabList.load</code>(**`path`**)

Load previously created vocablist object (containing all vocab objects for the dataset)

In [ ]:
vocab_list_1K = EhrVocabList.create(PATH_1K)

In [ ]:
vocab_list_1K.save()

Saved vocab lists to /home/vinod/.lemonpie/datasets/synthea/1K/processed


**Tests**

In [ ]:
vl_1K = EhrVocabList.load(PATH_1K)
obs_vocab, alg_vocab, crpl_vocab, med_vocab, img_vocab, proc_vocab, cnd_vocab, imm_vocab = vl_1K.records_vocabs
bday, bmonth, byear, marital, race, ethnicity, gender, birthplace, city, state, zipcode  = vl_1K.demographics_vocabs

**`records_vocabs`**

In [ ]:
obs_vocab.vocab_size

536

In [ ]:
proc_vocab.numericalize(['xxnone','65200003','428191000124101'])

[0, 67, 1]

In [ ]:
img_vocab.numericalize(['xxnone',344001])

[0, 8]

In [ ]:
proc_vocab.numericalize(['65200003']), proc_vocab.numericalize([65200003])

([67], [67])

In [ ]:
img_vocab.textify([0,1,2,3,4,5])

[('xxnone', 'Nothing recorded'),
 ('xxunk', 'Unknown'),
 ('40983000', {'Arm'}),
 ('51299004', {'Clavicle'}),
 ('8205005', {'Wrist'}),
 ('72696002', {'Knee'})]

In [ ]:
img_vocab.numericalize(['xxnone','xxunk', 51299004,51185008,12921003]) 

[0, 1, 3, 7, 9]

In [ ]:
obs_vocab.textify([0,1,2,3,4,5])

[('xxnone', 'Nothing recorded'),
 ('xxunk||xxunk||xxunk||xxunk', 'Unknown'),
 ('8302-2||45.1||cm||numeric', 'Body Height'),
 ('8302-2||83.5||cm||numeric', 'Body Height'),
 ('8302-2||121.9||cm||numeric', 'Body Height'),
 ('8302-2||160.29999999999998||cm||numeric', 'Body Height')]

In [ ]:
obs_vocab.textify([200])

[('20565-8||26.75||mmol/L||numeric', 'Carbon Dioxide')]

In [ ]:
obs_vocab.numericalize(['8302-2||200.3||cm||numeric', \
                            '72514-3||4||{score}||numeric', '10834-0||3.7||g/dL||numeric','29463-7||181.8||kg||numeric'])

[6, 9, 1, 16]

In [ ]:
obs_vocab.textify([50,150,250,300])

[('8310-5||38.724999999999994||Cel||numeric', 'Body temperature'),
 ('2085-9||65.05||mg/dL||numeric', 'High Density Lipoprotein Cholesterol'),
 ('6248-9||56.175000000000004||kU/L||numeric', 'Soybean IgE Ab in Serum'),
 ('33914-3||120.875||mL/min/{1.73_m2}||numeric',
  'Estimated Glomerular Filtration Rate')]

In [ ]:
med_vocab.textify([0,1,2,3,4])

[('xxnone', 'Nothing recorded'),
 ('xxunk', 'Unknown'),
 ('313782||START', {'Acetaminophen 325 MG Oral Tablet'}),
 ('748856||START', {'Yaz 28 Day Pack'}),
 ('1534809||START',
  {'168 HR Ethinyl Estradiol 0.00146 MG/HR / norelgestromin 0.00625 MG/HR Transdermal System'})]

In [ ]:
med_vocab.itoc[:5]

['xxnone', 'xxunk', '313782||START', '748856||START', '1534809||START']

In [ ]:
med_vocab.itoc[-10:]

['1873983||STOP',
 '1734340||STOP',
 '313185||STOP',
 '1650142||STOP',
 'xxamp',
 'xxamp',
 'xxamp',
 'xxamp',
 'xxamp',
 'xxamp']

In [ ]:
med_vocab.numericalize(['xxnone', 'xxunk', '834061||START','282464||START', '313782||START', '749882||START'])

[0, 1, 24, 1, 2, 66]

In [ ]:
med_vocab.numericalize(['834061||START'])

[24]

**`demographics_vocabs`**

In [ ]:
for vocab in vl_1K.demographics_vocabs:
    print(vocab.get_emb_dims())

(40, 8)
(16, 8)
(128, 8)
(8, 8)
(8, 8)
(8, 8)
(8, 8)
(248, 16)
(208, 16)
(8, 8)
(184, 16)


In [ ]:
bday.numericalize(['xxnone','xxunk', 1,10,31])

[0, 1, 2, 11, 32]

In [ ]:
bday.textify([0, 1, 2, 11, 32])

['xxnone', 'xxunk', '1', '10', '31']

In [ ]:
bmonth.textify([13])

['12']

In [ ]:
byear.numericalize(['1942',1947,])

[44, 49]

In [ ]:
byear.numericalize([1948])

[50]

In [ ]:
marital.vocab_size

8

In [ ]:
marital.ctoi

{'xxnone': 0, 'xxunk': 1, 'M': 2, 'xxxnan': 3, 'S': 4, 'xxamp': 7}

In [ ]:
marital.textify([0,1,2,3,4,5])

['xxnone', 'xxunk', 'M', 'xxxnan', 'S', 'xxamp']

In [ ]:
race.vocab_size

8

In [ ]:
race.textify([0,1,2,3,4])

['xxnone', 'xxunk', 'black', 'white', 'asian']

In [ ]:
race.ctoi

{'xxnone': 0,
 'xxunk': 1,
 'black': 2,
 'white': 3,
 'asian': 4,
 'native': 5,
 'other': 6,
 'xxamp': 7}

In [ ]:
vl_1K.age_mean, vl_1K.age_std

(16313.340455840456, 9600.296817631992)

## Get All Embedding Dimensions

In [ ]:
#export
def get_all_emb_dims(EhrVocabList, αd=0.5736):
    '''Get embedding dimensions for all vocab objects of the dataset'''
    demographics_dims = [vocab.get_emb_dims(αd) for vocab in EhrVocabList.demographics_vocabs]
    recs_dims          = [vocab.get_emb_dims(αd) for vocab in EhrVocabList.records_vocabs]
    
    demographics_dims_width = recs_dims_width = 0
    for emb_dim in demographics_dims:
        demographics_dims_width += emb_dim[1]
    for emb_dim in recs_dims:
        recs_dims_width += emb_dim[1]
        
    return demographics_dims, recs_dims, demographics_dims_width, recs_dims_width

In [ ]:
demographics_dims, recs_dims, demographics_dims_width, recs_dims_width = get_all_emb_dims(EhrVocabList.load(PATH_1K))

In [ ]:
demographics_dims

[(40, 8),
 (16, 8),
 (128, 8),
 (8, 8),
 (8, 8),
 (8, 8),
 (8, 8),
 (248, 16),
 (208, 16),
 (8, 8),
 (184, 16)]

In [ ]:
demographics_dims

[(40, 8),
 (16, 8),
 (128, 8),
 (8, 8),
 (8, 8),
 (8, 8),
 (8, 8),
 (248, 16),
 (208, 16),
 (8, 8),
 (184, 16)]

In [ ]:
recs_dims

[(536, 16), (32, 8), (56, 8), (232, 16), (16, 8), (144, 8), (184, 16), (24, 8)]

In [ ]:
demographics_dims_width, recs_dims_width

(112, 88)

In [ ]:
demographics_dims_width, recs_dims_width

(112, 88)

`αd` - a hyperparameter to control the width of the embedding matrices
- higher values result in wider embedding matrices

In [ ]:
demographics_dims, recs_dims, demographics_dims_width, recs_dims_width = get_all_emb_dims(EhrVocabList.load(PATH_1K), αd=10)

In [ ]:
demographics_dims

[(40, 152),
 (16, 120),
 (128, 200),
 (8, 104),
 (8, 104),
 (8, 104),
 (8, 104),
 (248, 240),
 (208, 224),
 (8, 104),
 (184, 224)]

In [ ]:
recs_dims

[(536, 288),
 (32, 144),
 (56, 160),
 (232, 232),
 (16, 120),
 (144, 208),
 (184, 224),
 (24, 136)]

In [ ]:
demographics_dims_width, recs_dims_width

(1680, 1512)

## Export -

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_basics.ipynb.
Converted 01_preprocessing_clean.ipynb.
Converted 02_preprocessing_vocab.ipynb.
Converted 03_preprocessing_transform.ipynb.
Converted 04_data.ipynb.
Converted 05_metrics.ipynb.
Converted 06_learn.ipynb.
Converted 07_models.ipynb.
Converted 08_experiment.ipynb.
Converted 998_coherent_clean.ipynb.
Converted 999_amp_testing.ipynb.
Converted 99_quick_walkthru.ipynb.
Converted 99_running_exps.ipynb.
Converted index.ipynb.
